# Notebook for Serving The Trained Model for Classifying TinyImageNet. Notebook (4/4) in the End-to-End Scalable Deep Learning Pipeline on Hops.


This notebook will send inference requests to a model serving instance that was exported by notebook number three ([Notebook number three](./Step3_Distributed_Training.ipynb)). This assumes that you have created a model serving instance of the model by using the hopsworks UI. You can find documentation on how to do this [here](https://hops.readthedocs.io/en/0.9/hopsml/model_serving.html)

## Imports

In [1]:
%local
import requests
from hops import util
from hops import hdfs
from hops import constants
import os
from hops import featurestore
import tensorflow as tf
import json
from hops import serving
import numpy as np

## Constants

In [2]:
%local
MODEL = "icebergmodel"
PROJECT = hdfs.project_name()
INFERENCE_URL = ("/" + 
                 constants.REST_CONFIG.HOPSWORKS_REST_RESOURCE + 
                 "/project/" + 
                 str(hdfs.project_id()) + 
                 "/inference/models/" + 
                 MODEL + 
                 ":predict"
                )
SERVER = ""
WORK_DIR = ""
CONCURRENCY = ""
NUM_TESTS = ""
TEST_DATASET = "train_tfrecords_iceberg_classification_dataset"
HEIGHT = 75
WIDTH = 75
CHANNELS = 3
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 10000
INPUT_SHAPE = 16875
NUM_CLASSES = 200

In [3]:
%local
def get_tf_dataset():
    tf_record_schema = featurestore.get_training_dataset_tf_record_schema(TEST_DATASET)
    dataset_dir = featurestore.get_training_dataset_path(TEST_DATASET)
    input_files = tf.gfile.Glob(dataset_dir + "/part-r-*")
    dataset = tf.data.TFRecordDataset(input_files)

    def decode(example_proto):
        name_list = ["band_1", "band_2", "band_avg", "is_iceberg"]
        example = tf.parse_single_example(example_proto, tf_record_schema)
        x = tf.stack([example[name_list[0]], example[name_list[1]], example[name_list[2]]], axis=1)
        x = tf.reshape(x, [75, 75, 3])
        y = [tf.cast(example[name_list[3]], tf.float32)]
        return x,y
    
    dataset = dataset.map(decode)
    return dataset

In [4]:
%local
def do_inference():
    with tf.Session() as sess:
        dataset = get_tf_dataset()
        dataset_iter = dataset.make_one_shot_iterator()
        next_element = dataset_iter.get_next()
        for i in range(10):
            x,y = sess.run(next_element)
            request_data={}
            request_data['instances'] = [x.tolist()]
            response = serving.make_inference_request("icebergmodel", data=request_data, verb= ":predict")
            print("prediction: {}, is_iceberg: {}".format(response['predictions'][0][0], y[0]))       

In [7]:
%local
do_inference()

prediction: 0.457645357, is_iceberg: 0.0
prediction: 0.457955331, is_iceberg: 0.0
prediction: 0.469079971, is_iceberg: 0.0
prediction: 0.458596766, is_iceberg: 1.0
prediction: 0.458393067, is_iceberg: 1.0
prediction: 0.459837317, is_iceberg: 1.0
prediction: 0.456256419, is_iceberg: 0.0
prediction: 0.457359701, is_iceberg: 0.0
prediction: 0.457865953, is_iceberg: 0.0
prediction: 0.458710402, is_iceberg: 0.0
